In [3]:
import os
import numpy as np
import rasterio
from rasterio.mask import mask
import openeo
import json
import time
from datetime import datetime, timedelta
import requests
import subprocess
import os
import json
import openeo
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

In [4]:
# Conectando ao serviço OpenEO
# Aqui, precisei fazer login no site. Isso me permitiu autenticar
connection = openeo.connect("openeo.dataspace.copernicus.eu")
connection.authenticate_oidc_device(store_refresh_token=True)

Visit https://identity.dataspace.copernicus.eu/auth/realms/CDSE/device?user_code=CGQP-QJHU 📋 to authenticate.

✅ Authorized successfully

<Connection to 'https://openeo.dataspace.copernicus.eu/openeo/1.2/' with OidcBearerAuth>

In [5]:
# Para definirmos as coleções que podemos usar
collections = connection.list_collections()
for collection in collections:
    print(collection['id'])

SENTINEL3_OLCI_L1B
SENTINEL3_SLSTR
SENTINEL_5P_L2
COPERNICUS_VEGETATION_PHENOLOGY_PRODUCTIVITY_10M_SEASON1
COPERNICUS_VEGETATION_PHENOLOGY_PRODUCTIVITY_10M_SEASON2
COPERNICUS_PLANT_PHENOLOGY_INDEX
ESA_WORLDCOVER_10M_2020_V1
ESA_WORLDCOVER_10M_2021_V2
COPERNICUS_VEGETATION_INDICES
SENTINEL2_L1C
SENTINEL2_L2A
SENTINEL1_GRD
COPERNICUS_30
LANDSAT8_L2
SENTINEL3_SYN_L2_SYN
SENTINEL3_SLSTR_L2_LST
SENTINEL1_GLOBAL_MOSAICS


Vamos selecionar a SENTINEL2_L2A, que realiza correção atmosférica.  
COPERNICUS/S2 (L1C): São produtos de reflectância no topo da atmosfera (TOA) sem correção atmosférica.    

COPERNICUS/S2_SR (L2A): São produtos de reflectância de superfície (SR) que já passaram por correção atmosférica. Estas imagens são processadas com o algoritmo Sen2Cor, que remove os efeitos atmosféricos.

In [ ]:
# Para saber o nome das bandas a extrair
satelite = 'SENTINEL2_L2A' # Diga aqui o satélite de interesse
collection_details = connection.describe_collection(satelite)

# Lista as bandas disponíveis
bands = [band['name'] for band in collection_details['summaries']['eo:bands']]
print(f"Bandas disponíveis na coleção {satelite}:") 
print(bands)

Bandas disponíveis na coleção SENTINEL2_L2A:
['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12', 'WVP', 'AOT', 'SCL', 'sunAzimuthAngles', 'sunZenithAngles', 'viewAzimuthMean', 'viewZenithMean']
